In [84]:
import numpy as np 
import pandas as pd 
import csv
import string
import scipy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from sklearn.metrics import confusion_matrix

In [40]:
TRAINING_FILE = 'train.csv'
TEST_FILE = 'test.csv'

In [41]:
def parse_train_data(TRAINING_FILE):
    with open(TRAINING_FILE) as file:
        csv_reader = csv.reader(file, delimiter=",")
        next(csv_reader)
        labels = []  # create labels list
        images = [] # create images list
        for row in csv_reader:
            labels.append(float(row[0]))
            image = np.array(row[1:]).astype(float)
            image = np.reshape(image, (28, 28)) #reshaping the image data
            images.append(image)
         # return numpy array
        return np.array(images).astype(np.float32), np.array(labels).astype(np.int32)

In [42]:
def parse_test_data(TEST_FILE):
    with open(TEST_FILE) as file:
        csv_reader = csv.reader(file, delimiter=",")
        next(csv_reader)
        images = []
        for row in csv_reader:
            image = np.array(row).astype(float)
            image = np.reshape(image, (28, 28))
            images.append(image)

        return np.array(images).astype(np.float32)

In [43]:
class StopAtAccuracy(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('sparse_categorical_accuracy') > 0.9999:
            print("\nReached 99.99% accuracy, stopping training!")
            self.model.stop_training = True

In [44]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.0001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

In [45]:
train_images, train_labels = parse_train_data(TRAINING_FILE)
train_images = np.expand_dims(train_images, axis=3)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow(
    x=train_images,
    y=train_labels,
    batch_size=32,
    shuffle=True
)

stop_at_accuracy_callback = StopAtAccuracy()

history = model.fit(
    train_generator,
    epochs=50,
    verbose=1,
    callbacks=[stop_at_accuracy_callback]
)


Epoch 1/50
1313/1313 [==============================] - 9s 7ms/step - loss: 0.3570 - sparse_categorical_accuracy: 0.8948
Epoch 2/50
1313/1313 [==============================] - 9s 7ms/step - loss: 0.2143 - sparse_categorical_accuracy: 0.9358
Epoch 3/50
1313/1313 [==============================] - 9s 7ms/step - loss: 0.1856 - sparse_categorical_accuracy: 0.9446
Epoch 4/50
1313/1313 [==============================] - 10s 7ms/step - loss: 0.1694 - sparse_categorical_accuracy: 0.9494
Epoch 5/50
1313/1313 [==============================] - 10s 8ms/step - loss: 0.1580 - sparse_categorical_accuracy: 0.9522
Epoch 6/50
1313/1313 [==============================] - 9s 7ms/step - loss: 0.1514 - sparse_categorical_accuracy: 0.9545
Epoch 7/50
1313/1313 [==============================] - 9s 7ms/step - loss: 0.1432 - sparse_categorical_accuracy: 0.9568
Epoch 8/50
1313/1313 [==============================] - 9s 7ms/step - loss: 0.1396 - sparse_categorical_accuracy: 0.9584
Epoch 9/50
1313/1313 [========

In [74]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# Save model
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Saving the model as mnist.h5


In [98]:
Model =tf.keras.models.load_model('mnist.h5')



In [113]:
y_pred = model.predict(test_images)
y_pred = [np.argmax(i) for i in y_pred]

875/875 [==============================] - 2s 2ms/step


In [126]:
val_y2 =test_images.reshape(-1)

In [134]:
r_labels=np.argmax(train_labels, axis=3)

AxisError: axis 3 is out of bounds for array of dimension 1

In [130]:
len(y_pred)

28000

In [132]:
matrix = confusion_matrix(r_labels, y_pred)

TypeError: Singleton array 11 cannot be considered a valid collection.

In [111]:
sns.heatmap(matrix, annot=True, cmap='Purples',fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('values')
plt.show()

NameError: name 'matrix' is not defined

In [69]:
test_images = parse_test_data(TEST_FILE)
test_images = np.expand_dims(test_images, axis=3)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow(
    x=test_images,
    batch_size=1,
    shuffle=False
)

predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

submission = pd.DataFrame({
    "ImageId": list(range(1, len(predicted_labels)+1)),
    "Label": predicted_labels
})

submission.to_csv("submission.csv", index=False)

28000/28000 [==============================] - 19s 673us/step
